In [ ]:
import h5py
import numpy as np
from datetime import datetime, timedelta
import pytz
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# Load the HDF5 file
file_path = '../data/Mobile.2024.12.03.5000.04.011.sensor.h5'
file_path_meta = '../data/Mobile.2024.12.03.5000.04.011.h5'

# Step 1: Load and extract data for all four limbs
limb_data = {}
with h5py.File(file_path, 'r') as h5_file:
    limb_data['Left Hand'] = h5_file['measurement/MetaWear Sensors Control/left_hand_data'][:]
    limb_data['Left Leg'] = h5_file['measurement/MetaWear Sensors Control/left_leg_data'][:]
    limb_data['Right Hand'] = h5_file['measurement/MetaWear Sensors Control/right_hand_data'][:]
    limb_data['Right Leg'] = h5_file['measurement/MetaWear Sensors Control/right_leg_data'][:]

# Step 2: Convert timestamps to actual date and time
epoch_start = datetime(1970, 1, 1)  # Unix epoch
formatted_data = {}
for limb, time_accel in limb_data.items():
    times = [epoch_start + timedelta(seconds=t) for t in time_accel[:, 0]]
    formatted_data[limb] = {
        "datetime": np.array(times),
        "acceleration": time_accel[:, 1]
    }

# Step 3: Adjust timestamps to Israel timezone
israel_tz = pytz.timezone("Asia/Jerusalem")
formatted_data_israel = {}
for limb, data in formatted_data.items():
    times_israel = [t.replace(tzinfo=pytz.utc).astimezone(israel_tz) for t in data["datetime"]]
    formatted_data_israel[limb] = {
        "datetime": np.array(times_israel),
        "acceleration": data["acceleration"]
    }

# Step 4: Create a unified timeline with 50 ms (0.05 seconds) period
start_time = max(data["datetime"][0] for data in formatted_data_israel.values())
end_time = min(data["datetime"][-1] for data in formatted_data_israel.values())
unified_timeline = np.arange(start_time.timestamp(), end_time.timestamp(), 0.05)

# Step 5: Interpolate the data for all limbs onto the unified timeline
interpolated_data = {}
for limb, data in formatted_data_israel.items():
    # Convert datetime to timestamps for interpolation
    timestamps = np.array([dt.timestamp() for dt in data["datetime"]])
    acceleration = data["acceleration"]
    
    # Create interpolation function
    interp_func = interp1d(timestamps, acceleration, kind='linear', fill_value="extrapolate")
    
    # Apply interpolation on the unified timeline
    interpolated_acceleration = interp_func(unified_timeline)
    interpolated_data[limb] = interpolated_acceleration

# Step 6: Plot the interpolated data
plt.figure(figsize=(12, 8))
unified_datetime = [datetime.fromtimestamp(ts, tz=israel_tz) for ts in unified_timeline]
for limb, interpolated_accel in interpolated_data.items():
    plt.plot(unified_datetime, interpolated_accel, label=limb)

plt.title("Interpolated Acceleration Data for Four Limbs")
plt.xlabel("Datetime (Israel Time)")
plt.ylabel("Acceleration (m/s²)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# create a new dataframe from interpolated_data and unified_datetime
interpolated_df = pd.DataFrame(interpolated_data, index=unified_datetime)
interpolated_df.index.name = 'datetime'
interpolated_df

In [ ]:
import h5py
import pandas as pd
from datetime import datetime
import pytz

# Load the HDF5 file

# Step 1: Extract the events table from the HDF5 file
events_data = None
with h5py.File(file_path_meta, 'r') as h5_file:
    events_data = h5_file['measurement/Task Management/events'][:]

# Step 2: Convert to a pandas DataFrame for better readability
events_table = pd.DataFrame(events_data, columns=["Event Name", "Event Type", "Timestamp"])

# Step 3: Decode byte strings to plain strings using map
events_table["Event Name"] = events_table["Event Name"].map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
events_table["Event Type"] = events_table["Event Type"].map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
events_table["Timestamp"] = events_table["Timestamp"].map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Step 4: Group by Event Name and organize into a new table with Start Time and End Time
events_grouped = events_table.groupby("Event Name")
manipulated_table = pd.DataFrame(columns=["Event Name", "Start Time", "End Time"])

for event_name, group in events_grouped:
    start_time = group[group["Event Type"] == "Start"]["Timestamp"].values
    end_time = group[group["Event Type"] == "End"]["Timestamp"].values
    manipulated_table = pd.concat([
        manipulated_table,
        pd.DataFrame({
            "Event Name": [event_name],
            "Start Time": start_time[0] if len(start_time) > 0 else None,
            "End Time": end_time[0] if len(end_time) > 0 else None
        })
    ], ignore_index=True)

# Step 5: Filter out "Task" and "Fixation" events
filtered_table = manipulated_table[
    ~manipulated_table["Event Name"].isin(["Task", "Fixation"])
]

# Step 6: Sort the table by Start Time
filtered_table_sorted = filtered_table.sort_values(by="Start Time").reset_index(drop=True)

# Step 7: Display the final table
print(filtered_table_sorted)


In [ ]:
# Convert Start Time and End Time to datetime
filtered_table_sorted['Start Time'] = pd.to_datetime(filtered_table_sorted['Start Time'])
filtered_table_sorted['End Time'] = pd.to_datetime(filtered_table_sorted['End Time'])

# Localize Start Time and End Time to the same timezone as interpolated_df index
israel_tz = pytz.timezone("Asia/Jerusalem")
filtered_table_sorted['Start Time'] = filtered_table_sorted['Start Time'].dt.tz_localize(israel_tz, ambiguous='NaT', nonexistent='NaT')
filtered_table_sorted['End Time'] = filtered_table_sorted['End Time'].dt.tz_localize(israel_tz, ambiguous='NaT', nonexistent='NaT')

# Create a dictionary to store the segmented dataframes
segmented_data = {}

# Iterate over each event and segment the interpolated_df
for index, row in filtered_table_sorted.iterrows():
    event_name = row['Event Name']
    start_time = row['Start Time']
    end_time = row['End Time']
    
    # Filter the interpolated_df based on the start and end time
    segment_df = interpolated_df[(interpolated_df.index >= start_time) & (interpolated_df.index <= end_time)]
    
    # Store the segmented dataframe in the dictionary
    segmented_data[event_name] = segment_df

# Print the segmented dataframes
for event_name, segment_df in segmented_data.items():
    print(f"Segment for {event_name}:")
    print(segment_df)
    print("\n")

In [ ]:

# Perform the calculation for all items in segmented_data
all_stats = {}

for event_name, data in segmented_data.items():
    slot_duration = pd.Timedelta(seconds=10)
    start_time = data.index.min()
    end_time = data.index.max()

    stats_list = []
    current_time = start_time
    while current_time < end_time:
        next_time = current_time + slot_duration
        slot_data = data[(data.index >= current_time) & (data.index < next_time)]
        
        if not slot_data.empty:
            stats = {
                'start_time': current_time,
                'end_time': next_time,
                'mean': slot_data.mean(),
                'std': slot_data.std(),
                'min': slot_data.min(),
                'max': slot_data.max()
            }
            stats_list.append(stats)
        
        current_time = next_time

    stats_df = pd.DataFrame(stats_list)
    all_stats[event_name] = stats_df

# Print the statistics for each event
for event_name, stats_df in all_stats.items():
    print(f"Statistics for {event_name}:")
    print(stats_df)
    print("\n")

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot mean values
axes[0, 0].set_title('Mean Acceleration')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Mean Acceleration')
for event_name, stats_df in all_stats.items():
    for limb in ['Left Hand', 'Left Leg', 'Right Hand', 'Right Leg']:
        axes[0, 0].plot(stats_df['start_time'], stats_df['mean'].apply(lambda x: x[limb]), label=f'{event_name} - {limb}')

# Plot standard deviation values
axes[0, 1].set_title('Standard Deviation of Acceleration')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Standard Deviation')
for event_name, stats_df in all_stats.items():
    for limb in ['Left Hand', 'Left Leg', 'Right Hand', 'Right Leg']:
        axes[0, 1].plot(stats_df['start_time'], stats_df['std'].apply(lambda x: x[limb]), label=f'{event_name} - {limb}')

# Plot minimum values
axes[1, 0].set_title('Minimum Acceleration')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Minimum Acceleration')
for event_name, stats_df in all_stats.items():
    for limb in ['Left Hand', 'Left Leg', 'Right Hand', 'Right Leg']:
        axes[1, 0].plot(stats_df['start_time'], stats_df['min'].apply(lambda x: x[limb]), label=f'{event_name} - {limb}')

# Plot maximum values
axes[1, 1].set_title('Maximum Acceleration')
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Maximum Acceleration')
for event_name, stats_df in all_stats.items():
    for limb in ['Left Hand', 'Left Leg', 'Right Hand', 'Right Leg']:
        axes[1, 1].plot(stats_df['start_time'], stats_df['max'].apply(lambda x: x[limb]), label=f'{event_name} - {limb}')

# Add legends
for ax in axes.flat:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()